In [1]:
import json

In [2]:
reddit_dict = dict()

with open('state_reddit_data.json') as f:
    reddit_dict = json.load(f)

In [3]:
# iterate through dict and only get the posts and comments
data = dict()

for k in reddit_dict.keys():
    state_posts = []
    state_comments = []
    
    for p in reddit_dict[k]:
        post = {
            "selftext": p["selftext"],
            "title": p["title"],
            "comments": [c["body"] for c in p["comments"]]
        }
        state_posts.append(post)
    data[k] = state_posts

In [4]:
# list of words to look for when filtering political posts
keywords = [
    "election",
    "president",
    "presidential",
    "democrat",
    "republican",
    "vote ",
    "voting",
    "donald",
    "trump",
    "donald trump",
    "kamala",
    "harris",
    "kamala harris",
    "abortion",
    "democracy",
    "immigration",
    "economy",
    "war ",
    "ukraine",
    "israel",
    "palestine",
    "climate",
    "healthcare",
    "inflation"
]

In [5]:
# filter for political posts
political_posts = dict()

for k in data.keys():
    posts = []

    for post in data[k]:
        for w in keywords:
            if w in post["selftext"].lower() or w in post["title"].lower():
                posts.append(post)

    political_posts[k] = posts

In [6]:
just_posts_size = dict()
for k in data.keys():
    just_posts_size[k] = len(political_posts[k])
just_posts_size

{'texas': 8,
 'california': 14,
 'newjersey': 0,
 'michigan': 5,
 'minnesota': 3,
 'colorado': 1,
 'wisconsin': 17,
 'florida': 5,
 'connecticut': 7,
 'oregon': 1,
 'ohio': 4,
 'hawaii': 0,
 'northcarolina': 16,
 'oklahoma': 11,
 'maryland': 3,
 'arizona': 3,
 'virginia': 15,
 'maine': 10,
 'indiana': 4,
 'iowa': 11,
 'washington': 10,
 'newhampshire': 3,
 'alaska': 9,
 'louisiana': 3,
 'massachusetts': 3,
 'vermont': 2,
 'newyork': 19,
 'arkansas': 5,
 'pennsylvania': 10,
 'alabama': 2,
 'kentucky': 6,
 'southcarolina': 1,
 'georgia': 7,
 'montana': 0,
 'delaware': 2,
 'utah': 6,
 'rhodeisland': 1,
 'missouri': 12,
 'tennessee': 10,
 'nebraska': 12,
 'illinois': 15,
 'westvirginia': 3,
 'newmexico': 7,
 'mississippi': 12,
 'kansas': 13,
 'northdakota': 11,
 'idaho': 9,
 'southdakota': 44,
 'wyoming': 6,
 'nevada': 8}

In [7]:
# filter for political comments in addition
political_data = dict()

for k in data.keys():
    posts_and_comments = []

    for post in data[k]:
        for w in keywords:
            if w in post["selftext"].lower() or w in post["title"].lower():
                posts_and_comments.append(post)
            elif w in " ".join(post["comments"]).lower():
                posts_and_comments.append(post)

    political_data[k] = posts_and_comments

In [8]:
posts_and_comments_size = dict()
for k in data.keys():
    posts_and_comments_size[k] = len(political_data[k])
posts_and_comments_size

{'texas': 171,
 'california': 70,
 'newjersey': 17,
 'michigan': 64,
 'minnesota': 68,
 'colorado': 55,
 'wisconsin': 88,
 'florida': 61,
 'connecticut': 41,
 'oregon': 78,
 'ohio': 105,
 'hawaii': 23,
 'northcarolina': 100,
 'oklahoma': 151,
 'maryland': 95,
 'arizona': 38,
 'virginia': 92,
 'maine': 123,
 'indiana': 109,
 'iowa': 144,
 'washington': 49,
 'newhampshire': 37,
 'alaska': 144,
 'louisiana': 79,
 'massachusetts': 62,
 'vermont': 38,
 'newyork': 175,
 'arkansas': 79,
 'pennsylvania': 152,
 'alabama': 124,
 'kentucky': 58,
 'southcarolina': 68,
 'georgia': 65,
 'montana': 13,
 'delaware': 57,
 'utah': 66,
 'rhodeisland': 15,
 'missouri': 122,
 'tennessee': 96,
 'nebraska': 126,
 'illinois': 162,
 'westvirginia': 77,
 'newmexico': 38,
 'mississippi': 100,
 'kansas': 79,
 'northdakota': 86,
 'idaho': 132,
 'southdakota': 247,
 'wyoming': 55,
 'nevada': 111}

In [9]:
political_comments = dict()

for k in political_data.keys():
    filtered_comments = []
    for state in political_data[k]:
        for c in state["comments"]:
               for w in keywords:
                   if w in c:
                       filtered_comments.append(c)
    political_comments[k] = filtered_comments

In [10]:
comments_size = dict()
for k in political_comments.keys():
    comments_size[k] = len(political_comments[k])
comments_size

{'texas': 3419,
 'california': 2072,
 'newjersey': 36,
 'michigan': 1163,
 'minnesota': 833,
 'colorado': 1198,
 'wisconsin': 1194,
 'florida': 521,
 'connecticut': 380,
 'oregon': 1414,
 'ohio': 2308,
 'hawaii': 50,
 'northcarolina': 3866,
 'oklahoma': 1194,
 'maryland': 2697,
 'arizona': 300,
 'virginia': 2719,
 'maine': 4302,
 'indiana': 3341,
 'iowa': 5392,
 'washington': 841,
 'newhampshire': 208,
 'alaska': 6842,
 'louisiana': 452,
 'massachusetts': 1311,
 'vermont': 162,
 'newyork': 9817,
 'arkansas': 951,
 'pennsylvania': 9018,
 'alabama': 2374,
 'kentucky': 640,
 'southcarolina': 1764,
 'georgia': 690,
 'montana': 34,
 'delaware': 674,
 'utah': 989,
 'rhodeisland': 41,
 'missouri': 2401,
 'tennessee': 1798,
 'nebraska': 2826,
 'illinois': 9529,
 'westvirginia': 1422,
 'newmexico': 511,
 'mississippi': 2646,
 'kansas': 1445,
 'northdakota': 3291,
 'idaho': 2106,
 'southdakota': 6655,
 'wyoming': 790,
 'nevada': 10842}

In [11]:
# DATA FORMAT
# political_data contains posts and comments of results when filtered for either political posts or comments

# political_data = {
#    'texas': [
#        {
#            'selftext': "__",
#            'title': "__",
#            'comments': ["__", "__", "__"]
#        },
#        {
#            'selftext': "__",
#            'title': "__",
#            'comments': ["__", "__", "__"]
#        },
#        ...
#    ],
#    'california': [
#        {
#            'selftext': "__",
#            'title': "__",
#            'comments': ["__", "__", "__"]
#        },
#        ...
#    ]
# }

# political_posts contains posts that contain any of the defined political keywords
# data should look the same as above

# political_comments contains state names mapped to comments that contain any of the defined political keywords
# political_comments = {
#    'texas': [__, __, __],
#    'california': [__, __, __],
#    ...
# }

In [12]:
# sentiment analysis
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [13]:
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

2024-11-24 00:02:26.537803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

In [15]:
## TEST - must use PyTorch version, the tensorflow one is not as accurate
text = "I love you!"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
print(ranking)
print(config.id2label)

for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

[2 1 0]
{0: 'negative', 1: 'neutral', 2: 'positive'}
1) positive 0.9749
2) neutral 0.0208
3) negative 0.0043


In [16]:
def get_sentiment_label_score(text):
    # get output from model
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    output = model(**encoded_input)

    # compute softmax scores
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # ranking = list of labels in decreasing order of sentiment score
    ranking = np.argsort(scores)[::-1]

    # return dictionary of {ranking: score} for given text
    # sentiment_map = dict()
    # for i in range(scores.shape[0]):
    #     l = config.id2label[ranking[i]]
    #     s = scores[ranking[i]]
    #     sentiment_map[l] = s

    # return largest score and the associated sentiment
    l = config.id2label[ranking[0]]
    s = scores[ranking[0]]
    return l, s

In [17]:
def record_sentiment(sent_data, score, key):
    if "scores" in sent_data.keys():
        sent_data["scores"].append(score)
    else:
        sent_data["scores"] = [score]

    if f"num_{key}" in sent_data.keys():
        sent_data[f"num_{key}"] += 1
    else:
        sent_data[f"num_{key}"] = 1

In [18]:
# Option 1: take sentiment of entire post, give an average rating

# Get the aggregated sentiment score for each post. Might want something like:
# all_sentiments = {
#    'texas': {
#         'positive': {
#            'scores': [0, 0, ...],
#            'average': 0.0,
#            'max': 0.0,
#            'min': 0.0,
#            'num_selftext': 0,
#            'num_title': 0
#        },
#        'neutral': {
#            ...
#        }
#        'negative': {
#            ...
#        },
#        'sentiment': 'positive'
#     }, ...}

# iterate through each state and compute highest weight sentiment for each post
all_sentiment = dict()

for k in political_posts.keys():
    state_data = {
        'positive': {},
        'neutral': {},
        'negative': {},
    }
    
    for post in political_posts[k]:
        label, score = get_sentiment_label_score(post['selftext'])
        record_sentiment(state_data[label], score, 'selftext')
    
        label, score = get_sentiment_label_score(post['title'])
        record_sentiment(state_data[label], score, 'title')
        
        all_sentiment[k] = state_data

In [19]:
all_sentiment['michigan']

{'positive': {'scores': [0.4012986, 0.4012986, 0.4012986], 'num_selftext': 3},
 'neutral': {'scores': [0.61926496,
   0.61926496,
   0.81732804,
   0.7474167,
   0.6769628],
  'num_title': 3,
  'num_selftext': 2},
 'negative': {'scores': [0.5218421, 0.6263394], 'num_title': 2}}

In [20]:
# get average, min, max, and dominant sentiment for each state
for k in all_sentiment.keys():
    avgs = dict()
    state_sent = all_sentiment[k]
    
    for s in state_sent.keys():
        sent = state_sent[s]
        if 'scores' not in sent.keys():
            sent['scores'] = [0]
            
        sent['min'] = min(sent['scores'])
        sent['max'] = max(sent['scores'])
        sent['average'] = np.mean(sent['scores'])
        avgs[sent['average']] = s
        
        if 'num_selftext' not in sent.keys():
            sent['num_selftext'] = 0
        if 'num_title' not in sent.keys():
            sent['num_title'] = 0

    avg_sentiment = max(avgs.keys())
    sentiment = avgs[avg_sentiment]
    all_sentiment[k]['avg_sentiment'] = avg_sentiment
    all_sentiment[k]['sentiment'] = sentiment